In [1]:
%tensorflow_version 2.x

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf


In [ ]:
##!wget https://www.dropbox.com/s/kfye28it99g9j12/imdb_labelled.txt

In [3]:
file=r'/content/train.csv'

In [4]:
import pandas as pd
imdb = pd.read_csv(file)

In [5]:
imdb.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0.0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0.0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0.0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0.0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0.0


In [ ]:
# you can download glove embeddings from here directly using wget
# just copy the link for the embeddings that you want to use 
# https://nlp.stanford.edu/projects/glove/

In [6]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip
# downloading and unzipping will take some time 

--2020-11-23 07:56:51--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2020-11-23 07:56:51--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2020-11-23 07:56:52--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [7]:
!unzip /content/glove.42B.300d.zip

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [8]:
!rm /content/glove.42B.300d.zip

In [9]:
import numpy as np

In [10]:
embeding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()
# this again will take good amount of time

In [11]:
from sklearn.model_selection import train_test_split
data_train,data_test=train_test_split(imdb,test_size=0.2)

In [12]:
x_train=data_train['question_text']
y_train=data_train['target']
x_test=data_test['question_text']
y_test=data_test['target']

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM,Activation,Dense,Input,Embedding,Dropout
from tensorflow.keras.models import Model

In [14]:
from nltk import word_tokenize

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
x_train

448038     How does a INFJ know when they have completed ...
919345     What is the correct saying, " the wisdom of So...
725775               When did you start to find your own VA?
1106351                        What yoga mat should you get?
157806     Are there examples of pseudo-science litterature?
                                 ...                        
984012     Have you ever been lost and asked the wrong pe...
1093352    In Top Gun, why did the jets run out of fuel s...
1275202    How do I know if I am not editing my photos to...
429209     What are the chemical properties of potassium ...
624506     How will you handle strategies that aren't wor...
Name: question_text, Length: 1032098, dtype: object

In [17]:
y = x_train.head()

In [18]:
for index,element in y.items():
  print(index)

448038
919345
725775
1106351
157806


In [19]:
sent_lens = []
errored_sents = []
for index,element in x_train.items():
    try:
        sent_lens.append(len(word_tokenize(element)))
    except:
        
        errored_sents.append(index)
    


#sent_lens=[len(word_tokenize(x)) for x in x_train]

In [20]:
errored_sents

[1290122]

In [21]:
np.percentile(sent_lens,95)

31.0

In [22]:
type(x_train)

pandas.core.series.Series

In [23]:
x_train = x_train.drop(1290122,axis=0)
y_train = y_train.drop(1290122,axis=0)

In [24]:
max_len=31

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [25]:
seq_train_matrix.shape,seq_test_matrix.shape,vocab_size

((1032097, 31), (258025, 31), 194804)

In [26]:
# creating our own embedding matrix to bring down the size to 300
# we'll use 300 D vector representation of the words from pretrained embedding index 
# that we downloaded 

embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector
        
# if there are specific words which are not present in pretrained embedding 
# their weights will remain 0. if there are too many such words 
# then you should probably not use pretrained embeddings 

In [27]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.20838   , -0.14932001, -0.017528  , ..., -0.54066002,
         0.21199   , -0.0094357 ],
       [ 0.13877   , -0.0021201 , -0.091039  , ...,  0.024614  ,
         0.46514001,  0.11363   ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.003073  ,  0.19486   , -0.1319    , ..., -0.23131   ,
         0.35293999, -0.2428    ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [28]:
# Now while creating the model for text data , we'll keep embedding layer weights 
# non-trainable as we are supplying those values from our end 

inputs=Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer=LSTM(50)(embed)
dense1=Dense(10,activation='relu')(lstm_layer)
drop=Dropout(0.2)(dense1)
final_layer=Dense(1,activation='sigmoid')(drop)

model=Model(inputs=inputs,outputs=final_layer)

In [29]:
# you can see that bulk of the weights come from embedding layer
# using pretrained weights for the same , brings down number of trainable parameters 
# by a huge amount. This in turn results in pretty short training times 
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      [(None, 31)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 31, 300)           58441500  
_________________________________________________________________
lstm (LSTM)                  (None, 50)                70200     
_________________________________________________________________
dense (Dense)                (None, 10)                510       
_________________________________________________________________
dropout (Dropout)            (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 58,512,221
Trainable params: 70,721
Non-trainable params: 58,441,500
_____________________________________

In [30]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [31]:
y_train.value_counts(),37981*12

(0.0    968238
 1.0     63859
 Name: target, dtype: int64, 455772)

In [32]:
model.fit(seq_train_matrix,y_train.values,validation_data=[seq_test_matrix,y_test.values],epochs=40,class_weight={0:1,1:12},batch_size=1024)

Epoch 1/40
1008/1008 [==============================] - 87s 86ms/step - loss: 0.5216 - accuracy: 0.8844 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/40
1008/1008 [==============================] - 89s 88ms/step - loss: 0.4528 - accuracy: 0.8995 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/40
1008/1008 [==============================] - 86s 85ms/step - loss: 0.4252 - accuracy: 0.9041 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/40
1008/1008 [==============================] - 86s 85ms/step - loss: 0.4041 - accuracy: 0.9063 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/40
1008/1008 [==============================] - 87s 86ms/step - loss: 0.3872 - accuracy: 0.9075 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/40
1008/1008 [==============================] - 86s 85ms/step - loss: 0.3689 - accuracy: 0.9096 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/40
1008/1008 [==============================] - 86s 86ms/step - loss

In [33]:
p=model.predict(seq_test_matrix)

In [34]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,p)

0.9531809904554669